<a href="https://colab.research.google.com/github/manasdhote/Capston_Project/blob/main/Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh panel --quiet

import pandas as pd
import pathway as pw
import datetime
import panel as pn
pn.extension()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load dataset
df = pd.read_csv("dataset.csv")

# Combine date and time into Timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# Sort by time
df = df.sort_values('Timestamp').reset_index(drop=True)

# Map vehicle weights
vehicle_map = {"car": 1.0, "bike": 0.8, "truck": 1.5 ,"cycle": 0.5}
df['vehicle'] = df['VehicleType'].map(vehicle_map).fillna(1.0)

# Map traffic nearby
traffic_map = {"low": 0.0, "average": 0.5, "high": 1.0}
df['Traffic'] = df['TrafficConditionNearby'].map(traffic_map).fillna(0.0) # Fill NaN with 0.0

# Select columns for streaming
stream_df = df[["Timestamp", "SystemCodeNumber", "Capacity","Occupancy", "IsSpecialDay", "vehicle","QueueLength","Traffic"]]

# Save to CSV
stream_df.to_csv("parking_stream.csv", index=False)

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp:         str
    SystemCodeNumber:  str
    Capacity:          float
    Occupancy:         float
    IsSpecialDay:      int
    vehicle:           float
    QueueLength:       float
    Traffic:           float

source = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=500  # Simulated rows/second
)

# Parse datetime
source = source.with_columns(
    t = source.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)


In [ ]:
a          = 1.0
b          = 0.7
c          = 0.5
d          = 0.3
e          = 0.3
f          = 0.2
base_price = 10

price = (
    source
    .windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        window_end = pw.this._pw_window_end,
        max_occupancy  = pw.reducers.max(pw.this.Occupancy),
        min_occupancy  = pw.reducers.min(pw.this.Occupancy),
        capacity       = pw.reducers.max(pw.this.Capacity),
        is_special     = pw.reducers.max(pw.this.IsSpecialDay),
        vehicle        = pw.reducers.max(pw.this.vehicle),
        queue_len      = pw.reducers.max(pw.this.QueueLength),
        traffic        = pw.reducers.max(pw.this.Traffic),
        system_id      = pw.reducers.max(pw.this.SystemCodeNumber),
    )
    .with_columns(
        demand = (
            a   * (pw.this.max_occupancy + pw.this.min_occupancy)/(2 * pw.this.capacity)
            + b  * pw.this.queue_len
            + c * pw.this.traffic
            + d * pw.this.is_special
            + e * pw.this.vehicle
        )
    )
    .with_columns(
        Price = base_price * (1 + f * pw.this.demand)
    )
    .select(
        SystemCodeNumber = pw.this.system_id,
        Timestamp        = pw.this.window_end,
        vehicle          = pw.this.vehicle,
        demand           = pw.this.demand,
        Price            = pw.this.Price
    )
)


In [ ]:
pw.io.csv.write(
    table=price,
    filename="price_output_2.csv"
)

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [ ]:
import panel as pn
import bokeh.plotting as bp
from bokeh.models import ColumnDataSource

pn.extension()

#  Streaming plot function for real-time price updates
def price_plotter(source: ColumnDataSource):
    fig = bp.figure(
        height=400,
        width=800,
        title="Real-Time Dynamic Parking Price",
        x_axis_type="datetime",
        x_axis_label="Timestamp",
        y_axis_label="Price"
    )

    # Line + dot markers
    fig.line("Timestamp", "Price", source=source, line_width=2, color="red")
    fig.scatter("Timestamp", "Price", source=source, size=5, color="navy")

    return fig

#  Connect your demand-based price table to Bokeh plot
viz = price.plot(price_plotter, sorting_col="Timestamp")

# ⬇ Show the visualization using Panel
pn.Column(viz).servable()



Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
pw.run()


Output()